# Phase 2 — Feature Engineering (Rolling & Team Form Fix)

**Goal:** Add rolling form, reliability (DNF rate), team rolling points, and track history to your season table using index-safe operations.

**How to use:** Run each step **in order**. Each step has a short goal and a 2‑line explanation. When it finishes, move to the next step.

## Step 0 — Load `season_df` (from memory or from `/data/processed`)

**Goal:** Ensure we have `season_df` ready. If it's not defined, we load the latest `season_table_fast_robust_*.csv` (or `season_table_fast_*.csv`) from `/data/processed`.

**Why:** Rolling features require a season-wide table. This step makes the notebook self-contained.

In [1]:
from pathlib import Path
import pandas as pd, numpy as np
import glob

# Project paths
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PROCESSED = ROOT / "data" / "processed"
PROCESSED.mkdir(parents=True, exist_ok=True)

# If season_df already exists (e.g., defined in your previous notebook), keep it.
try:
    _check = season_df.head(1)  # type: ignore[name-defined]
    print("Using season_df from memory:", season_df.shape)  # type: ignore[name-defined]
except Exception:
    # Try to load the most recent robust season table
    candidates = sorted([*PROCESSED.glob("season_table_fast_robust_*_k*.csv"),
                         *PROCESSED.glob("season_table_fast_*_k*.csv")],
                        key=lambda p: (p.stat().st_mtime, p.name), reverse=True)
    if not candidates:
        raise FileNotFoundError("No season_table_*.csv found in /data/processed. Run the season builder first.")
    season_path = candidates[0]
    season_df = pd.read_csv(season_path)
    # Parse dates if present
    if "event_date" in season_df.columns:
        season_df["event_date"] = pd.to_datetime(season_df["event_date"], errors="coerce")
    print("Loaded:", season_path.name, "| shape:", season_df.shape)

# Small sanity: make sure required columns exist
req_cols = ["Driver","TeamName","finish_pos","points","Status","event_name","event_date","group_key","year"]
missing = [c for c in req_cols if c not in season_df.columns]
if missing:
    raise ValueError(f"Missing columns in season_df: {missing}. Rebuild the season table with the robust builder.")

Loaded: season_table_fast_robust_2023_k10.csv | shape: (200, 18)


## Step 1 — Base copy & DNF flag

**Goal:** Copy the season table, sort by driver/date, and add a conservative `dnf` flag for reliability features.

**Why:** We keep the original data intact and compute a robust DNF signal for rolling stats.

In [2]:
df = season_df.copy().sort_values(["Driver","event_date"]).reset_index(drop=True)

def is_dnf_row(row):
    status = str(row.get("Status", "")).lower()
    pos = row.get("finish_pos", np.nan)
    pts = row.get("points", 0.0)
    return (("finished" not in status) and ("classified" not in status) and (pd.isna(pos) or pos > 20)) or (pd.isna(pos) and pts == 0)

df["dnf"] = df.apply(is_dnf_row, axis=1).astype(int)
print("Rows:", len(df), "| Drivers:", df['Driver'].nunique(), "| Teams:", df['TeamName'].nunique())

Rows: 200 | Drivers: 20 | Teams: 10


## Step 2 — Rolling form per driver (index-safe)

**Goal:** Add rolling averages of finish position & points, plus rolling DNF rate using `groupby(...).transform(...)`.

**Why:** `transform` keeps row alignment 1:1, avoiding index mismatches and future pandas warnings.

In [3]:
df = df.sort_values(["Driver","event_date"]).reset_index(drop=True)

df["roll3_avg_pos"]  = df.groupby("Driver")["finish_pos"].transform(lambda s: s.rolling(3, min_periods=1).mean())
df["roll5_avg_pos"]  = df.groupby("Driver")["finish_pos"].transform(lambda s: s.rolling(5, min_periods=1).mean())
df["roll3_pts"]      = df.groupby("Driver")["points"].transform(lambda s: s.rolling(3, min_periods=1).mean())
df["roll5_pts"]      = df.groupby("Driver")["points"].transform(lambda s: s.rolling(5, min_periods=1).mean())
df["roll5_dnf_rate"] = df.groupby("Driver")["dnf"].transform(  lambda s: s.rolling(5, min_periods=1).mean())

print("Added driver rolling features.")
df.head(3)

Added driver rolling features.


,Driver,TeamName,finish_pos,points,top10,Status,grid_pos,qual_pos,best_qual_t,delta_to_pole_s,...,round,event_name,event_date,group_key,dnf,roll3_avg_pos,roll5_avg_pos,roll3_pts,roll5_pts,roll5_dnf_rate
0,ALB,Williams,10.0,1.0,1,Finished,15.0,15.0,0 days 00:01:31.461000,1.753,...,1,Bahrain Grand Prix,2023-03-05,2023_R1,0,10.0,10.0,1.000000,1.000000,0.0
1,ALB,Williams,19.0,0.0,0,Retired,17.0,17.0,0 days 00:01:29.994000,1.729,...,2,Saudi Arabian Grand Prix,2023-03-19,2023_R2,0,14.5,14.5,0.500000,0.500000,0.0
2,ALB,Williams,19.0,0.0,0,Retired,8.0,8.0,0 days 00:01:17.609000,0.877,...,3,Australian Grand Prix,2023-04-02,2023_R3,0,16.0,16.0,0.333333,0.333333,0.0


## Step 3 — Team rolling points (one row per team per race → merge back)

**Goal:** Compute team points per race, then rolling means per team, and merge back to driver rows via `(TeamName, group_key)`.

**Why:** This avoids `groupby.apply` reindex issues and ensures each driver row gets the same team signal for that race.

In [4]:
# 1) Aggregate team points per race
team_race = (df.groupby(["TeamName","group_key","event_date"], as_index=False)["points"]
               .sum()
               .rename(columns={"points":"team_points_race"}))

# 2) Rolling team points over time
team_race = team_race.sort_values(["TeamName","event_date"]).reset_index(drop=True)
team_race["team_roll3_pts"] = (team_race.groupby("TeamName")["team_points_race"]
                               .transform(lambda s: s.rolling(3, min_periods=1).mean()))
team_race["team_roll5_pts"] = (team_race.groupby("TeamName")["team_points_race"]
                               .transform(lambda s: s.rolling(5, min_periods=1).mean()))

# 3) Merge back to driver rows
df = df.merge(team_race[["TeamName","group_key","team_roll3_pts","team_roll5_pts"]],
              on=["TeamName","group_key"], how="left")
print("Added team rolling features.")
df.head(3)

Added team rolling features.


,Driver,TeamName,finish_pos,points,top10,Status,grid_pos,qual_pos,best_qual_t,delta_to_pole_s,...,event_date,group_key,dnf,roll3_avg_pos,roll5_avg_pos,roll3_pts,roll5_pts,roll5_dnf_rate,team_roll3_pts,team_roll5_pts
0,ALB,Williams,10.0,1.0,1,Finished,15.0,15.0,0 days 00:01:31.461000,1.753,...,2023-03-05,2023_R1,0,10.0,10.0,1.000000,1.000000,0.0,1.000000,1.000000
1,ALB,Williams,19.0,0.0,0,Retired,17.0,17.0,0 days 00:01:29.994000,1.729,...,2023-03-19,2023_R2,0,14.5,14.5,0.500000,0.500000,0.0,0.500000,0.500000
2,ALB,Williams,19.0,0.0,0,Retired,8.0,8.0,0 days 00:01:17.609000,0.877,...,2023-04-02,2023_R3,0,16.0,16.0,0.333333,0.333333,0.0,0.333333,0.333333


## Step 4 — Track history per driver at this event (leakage-safe)

**Goal:** For each `(Driver, event_name)`, compute the average of **past** finishes using `shift().expanding().mean()`.

**Why:** The `shift()` excludes the current race, preventing leakage; `transform` keeps perfect index alignment.

In [5]:
df = df.sort_values(["Driver","event_name","event_date"]).reset_index(drop=True)
df["track_hist_avg_pos"] = (df.groupby(["Driver","event_name"])["finish_pos"]
                              .transform(lambda s: s.shift().expanding().mean()))
print("Added track history feature.")
df.head(3)

Added track history feature.


,Driver,TeamName,finish_pos,points,top10,Status,grid_pos,qual_pos,best_qual_t,delta_to_pole_s,...,group_key,dnf,roll3_avg_pos,roll5_avg_pos,roll3_pts,roll5_pts,roll5_dnf_rate,team_roll3_pts,team_roll5_pts,track_hist_avg_pos
0,ALB,Williams,19.0,0.0,0,Retired,8.0,8.0,0 days 00:01:17.609000,0.877,...,2023_R3,0,16.000000,16.0,0.333333,0.333333,0.0,0.333333,0.333333,NaN
1,ALB,Williams,11.0,0.0,0,Finished,10.0,10.0,0 days 00:01:05.387000,0.996,...,2023_R9,0,11.333333,12.4,2.000000,1.200000,0.0,2.000000,1.200000,NaN
2,ALB,Williams,12.0,0.0,0,Finished,12.0,13.0,0 days 00:01:41.818000,1.615,...,2023_R4,0,16.666667,15.0,0.000000,0.250000,0.0,0.000000,0.250000,NaN


## Step 5 — Save enriched season dataset

**Goal:** Persist the enriched features to `/data/processed` for modeling in Phase 3.

**Why:** Keeps your modeling inputs reproducible and decoupled from data assembly.

In [6]:
year_val = int(df["year"].dropna().iloc[0]) if "year" in df.columns and df["year"].notna().any() else 0
out_path = PROCESSED / (f"season_{year_val}_features_enriched.csv" if year_val else "season_features_enriched.csv")
df.to_csv(out_path, index=False)
print("Saved:", out_path.name, "| shape:", df.shape)

Saved: season_2023_features_enriched.csv | shape: (200, 27)


## Phase 2.5 Ensuring we have multiple races

## Check how many races we have (groups)

- Goal: verify we have enough race-weekends for CV; if not, rebuild quickly.
- We need ≥3 race-weekends (groups) for meaningful GroupKFold.
- If you see 1, run the next cell to build more rounds fast.

In [7]:
import pandas as pd
from pathlib import Path

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PROCESSED = ROOT / "data" / "processed"

df = pd.read_csv(PROCESSED / "season_2025_features_enriched.csv")
n_groups = df["group_key"].nunique()
print("Rows:", df.shape, "| race-weekends:", n_groups)


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\maxnd\\Documents\\Machine Learning\\f1-race-predictor\\data\\processed\\season_2025_features_enriched.csv'

## Building more rounds fast (K=10) and re-enrich
- Goal: create a bigger season set quickly, then re-run the rolling notebook afterwards.
- This builds K=10 rounds using results-only loads (fast), then re-applies rolling/team/track features.

In [ ]:
import logging, numpy as np, pandas as pd, fastf1
from pathlib import Path
logging.getLogger("fastf1").setLevel(logging.WARNING)

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PROCESSED = ROOT / "data" / "processed"

# Reuse fast, results-only loaders (defined earlier). If you don't have them in this notebook, paste them here again.
def load_results_fast(year:int, rnd:int, code:str):
    s = fastf1.get_session(year, rnd, code)
    try: s.load(laps=False, telemetry=False, weather=False, messages=False)
    except TypeError:
        try: s.load(telemetry=False)
        except Exception: s.load()
    return s

def extract_quali_df(year: int, rnd: int, rres_for_map: pd.DataFrame) -> pd.DataFrame:
    q = load_results_fast(year, rnd, "Q")
    qres = q.results.copy()
    num_to_abbr = {}
    if {"DriverNumber","Abbreviation"}.issubset(rres_for_map.columns):
        num_to_abbr = dict(zip(rres_for_map["DriverNumber"], rres_for_map["Abbreviation"]))
    if "Abbreviation" in qres.columns:
        qres = qres.rename(columns={"Abbreviation": "Driver"})
    elif "DriverNumber" in qres.columns and num_to_abbr:
        qres["Driver"] = qres["DriverNumber"].map(num_to_abbr)
    else:
        return pd.DataFrame(columns=["Driver","qual_pos","best_qual_t","delta_to_pole_s"])
    for c in ["Q1","Q2","Q3"]:
        if c in qres.columns: qres[c] = pd.to_timedelta(qres[c], errors="coerce")
    qres = qres.rename(columns={"Position":"qual_pos"})
    tcols = [c for c in ["Q1","Q2","Q3"] if c in qres.columns]
    if tcols:
        qres["best_qual_t"] = qres[tcols].min(axis=1, skipna=True)
        pole = qres["best_qual_t"].min()
        qres["delta_to_pole_s"] = (qres["best_qual_t"] - pole).dt.total_seconds()
    else:
        qres["best_qual_t"] = pd.NaT; qres["delta_to_pole_s"] = np.nan
    return qres[["Driver","qual_pos","best_qual_t","delta_to_pole_s"]]

# Light practice (uses your cached CSVs if present; otherwise returns empty safely)
def practice_features_light(year:int, rnd:int) -> pd.DataFrame:
    RAW = PROCESSED.parent / "raw"
    def read_laps(y,r,code):
        p = RAW / f"laps_{y}_R{r}_{code}.csv"
        if not p.exists(): raise FileNotFoundError
        df = pd.read_csv(p)
        if "LapTime" in df.columns: df["LapTime"] = pd.to_timedelta(df["LapTime"], errors="coerce")
        return df
    frames = []
    for code in ["FP1","FP2","FP3"]:
        try:
            laps = read_laps(year, rnd, code)
        except FileNotFoundError:
            continue
        valid = laps.copy()
        for col in ["PitInLap","PitOutLap","IsAccurate"]:
            if col in valid.columns:
                if col in ["PitInLap","PitOutLap"]: valid = valid[~valid[col].fillna(False)]
                else: valid = valid[valid[col].fillna(True)]
        valid = valid[valid["LapTime"].notna()]
        if {"Driver","Stint"}.issubset(valid.columns):
            stint_sizes = valid.groupby(["Driver","Stint"])["LapTime"].transform("size")
            longrun = valid[stint_sizes >= 5]
        else:
            longrun = pd.DataFrame(columns=valid.columns)
        grp_all = valid.groupby("Driver")["LapTime"]
        grp_lr  = longrun.groupby("Driver")["LapTime"] if not longrun.empty else None
        dfp = pd.DataFrame({
            "Driver": grp_all.count().index,
            f"{code}_mean_all_s": grp_all.mean().dt.total_seconds().values,
            f"{code}_laps": grp_all.count().values,
        })
        med_all = grp_all.median().dt.total_seconds()
        med_lr  = (grp_lr.median().dt.total_seconds() if grp_lr is not None else pd.Series(dtype=float))
        dfp[f"{code}_median_longrun_s"] = dfp["Driver"].map(med_lr).fillna(dfp["Driver"].map(med_all))
        frames.append(dfp)
    if not frames: return pd.DataFrame(columns=["Driver"])
    out = frames[0]
    for f in frames[1:]: out = out.merge(f, on="Driver", how="outer")
    out["fp_mean_all_s"]      = out[[c for c in out.columns if c.endswith("_mean_all_s")]].mean(axis=1)
    out["fp_median_longrun_s"]= out[[c for c in out.columns if c.endswith("_median_longrun_s")]].mean(axis=1)
    out["fp_total_laps"]      = out[[c for c in out.columns if c.endswith("_laps")]].sum(axis=1).astype(int)
    return out[["Driver","fp_mean_all_s","fp_median_longrun_s","fp_total_laps"]]

def build_season_table_fast_robust(year:int, k:int=10) -> pd.DataFrame:
    schedule = fastf1.get_event_schedule(year, include_testing=False)
    rounds = schedule["RoundNumber"].tolist()[:k]
    rows = []
    for rnd in rounds:
        try:
            r = load_results_fast(year, rnd, "R")
            rraw = r.results.copy()
            keep = [c for c in ["Abbreviation","DriverNumber","TeamName","GridPosition","Position","Points","Status"] if c in rraw.columns]
            rres = rraw[keep].copy()
            base = rres.rename(columns={"Abbreviation":"Driver"})
            base["finish_pos"] = pd.to_numeric(base.get("Position"), errors="coerce")
            base["points"]     = pd.to_numeric(base.get("Points"), errors="coerce").fillna(0.0)
            base["top10"]      = (base["points"] > 0).astype(int)
            grid = base[["Driver","GridPosition"]].rename(columns={"GridPosition":"grid_pos"}) if "GridPosition" in base.columns else pd.DataFrame(columns=["Driver","grid_pos"])
            qres = extract_quali_df(year, rnd, rraw)
            fp   = practice_features_light(year, rnd)
            ev   = fastf1.get_event(year, rnd)
            merged = (base[["Driver","TeamName","finish_pos","points","top10","Status"]]
                      .merge(grid, on="Driver", how="left")
                      .merge(qres, on="Driver", how="left")
                      .merge(fp, on="Driver", how="left"))
            merged["year"], merged["round"] = year, rnd
            merged["event_name"], merged["event_date"] = ev["EventName"], pd.to_datetime(ev["EventDate"])
            merged["group_key"] = f"{year}_R{rnd}"
            rows.append(merged)
        except Exception as e:
            print(f"Skip {year} R{rnd}: {e.__class__.__name__}: {e}")
    out = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    out.to_csv(PROCESSED / f"season_table_fast_robust_{year}_k{k}.csv", index=False)
    return out

YEAR, K = 2025, 10
season_big = build_season_table_fast_robust(YEAR, K)
print("Built:", season_big.shape, "| groups:", season_big['group_key'].nunique())

# Re-use your rolling steps (R0–R4) or do the short inline version:
season_big["event_date"] = pd.to_datetime(season_big["event_date"], errors="coerce")
df = season_big.sort_values(["Driver","event_date"]).reset_index(drop=True)

def is_dnf_row(row):
    st = str(row.get("Status","")).lower()
    pos = row.get("finish_pos"); pts = row.get("points",0.0)
    return (("finished" not in st) and ("classified" not in st) and (pd.isna(pos) or pos>20)) or (pd.isna(pos) and pts==0)

df["dnf"] = df.apply(is_dnf_row, axis=1).astype(int)
df["roll3_avg_pos"]  = df.groupby("Driver")["finish_pos"].transform(lambda s: s.rolling(3, min_periods=1).mean())
df["roll5_avg_pos"]  = df.groupby("Driver")["finish_pos"].transform(lambda s: s.rolling(5, min_periods=1).mean())
df["roll3_pts"]      = df.groupby("Driver")["points"].transform(lambda s: s.rolling(3, min_periods=1).mean())
df["roll5_pts"]      = df.groupby("Driver")["points"].transform(lambda s: s.rolling(5, min_periods=1).mean())
df["roll5_dnf_rate"] = df.groupby("Driver")["dnf"].transform(  lambda s: s.rolling(5, min_periods=1).mean())

team_race = (df.groupby(["TeamName","group_key","event_date"], as_index=False)["points"].sum()
               .rename(columns={"points":"team_points_race"}))
team_race = team_race.sort_values(["TeamName","event_date"]).reset_index(drop=True)
team_race["team_roll3_pts"] = team_race.groupby("TeamName")["team_points_race"].transform(lambda s: s.rolling(3, min_periods=1).mean())
team_race["team_roll5_pts"] = team_race.groupby("TeamName")["team_points_race"].transform(lambda s: s.rolling(5, min_periods=1).mean())
df = df.merge(team_race[["TeamName","group_key","team_roll3_pts","team_roll5_pts"]], on=["TeamName","group_key"], how="left")

df = df.sort_values(["Driver","event_name","event_date"]).reset_index(drop=True)
df["track_hist_avg_pos"] = df.groupby(["Driver","event_name"])["finish_pos"].transform(lambda s: s.shift().expanding().mean())

out = PROCESSED / "season_2025_features_enriched.csv"
df.to_csv(out, index=False)
print("Re-saved enriched:", out.name, "| shape:", df.shape, "| groups:", df['group_key'].nunique())


req         WARNING 	DEFAULT CACHE ENABLED! (3.0 GB) C:\Users\maxnd\AppData\Local\Temp\fastf1


Built: (200, 18) | groups: 10
Re-saved enriched: season_2023_features_enriched.csv | shape: (200, 27) | groups: 10
